In [1]:
import ast
import json

import pandas as pd

In [2]:
# convert "['a','b']" to a set {'a', 'b'}
def to_set(x):
    list_string = json.loads(x)
    list_raw = ast.literal_eval(list_string)
    return set(list_raw)

In [3]:
# Read the CSV
output_df = pd.read_csv("data/_backup/FoodKG_ingredients_normalized.csv")
test_df = pd.read_csv("data/test/TEST_FoodKG_ingredients_normalized.csv")
# Rename the test column so we don't clash on merge
test_df_renamed = test_df.rename(columns={"ingredients_normalized": "ingredients_test"})
# Merge using test_df as the left df to keep its order
merged_df = test_df_renamed[["id", "ingredients_test"]].merge(
    output_df, on="id", how="inner"
)
merged_df.head()

,id,ingredients_test,title,ingredients,ingredients_normalized
0,0,"""['brown sugar', 'evaporated milk', 'vanilla',...",No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","""['brown sugar', 'evaporated milk', 'vanilla',..."
1,1,"""['chipped beef', 'chicken breast', 'cream of ...",Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...","""['chipped beef', 'chicken breast', 'cream of ..."
2,2,"""['corn', 'cream cheese', 'butter', 'garlic po...",Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","""['corn', 'cream cheese', 'butter', 'garlic po..."
3,3,"""['whole chicken', 'chicken gravy', 'cream of ...",Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","""['chicken', 'chicken gravy', 'mushroom soup',..."
4,4,"""['peanut butter', 'graham cracker', 'butter',...",Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...","""['peanut butter', 'graham cracker crumbs', 'b..."


In [4]:
output_sets = [to_set(x) for x in merged_df["ingredients_normalized"]]
test_sets = [to_set(x) for x in merged_df["ingredients_test"]]

new_df = pd.DataFrame({
    "id": merged_df["id"],
    "ingredients": merged_df["ingredients"],
    "output_set": output_sets,
    "test_set": test_sets,
})
new_df.head()

,id,ingredients,output_set,test_set
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","{brown sugar, evaporated milk, pecan, butter, ...","{brown sugar, evaporated milk, pecan, margarin..."
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","{chipped beef, cream of mushroom soup, chicken...","{chipped beef, cream of mushroom soup, chicken..."
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","{cream cheese, salt, pepper, corn, butter, gar...","{cream cheese, salt, corn, butter, black peppe..."
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","{mushroom soup, chicken gravy, stuffing, chick...","{cream of mushroom soup, chicken gravy, stuffi..."
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","{peanut butter, butter, powdered sugar, graham...","{graham cracker, peanut butter, chocolate chip..."


In [5]:
def precision_recall_f1_per_row(row):
    pred = row["output_set"]
    true = row["test_set"]

    tp = len(pred & true)
    fp = len(pred - true)
    fn = len(true - pred)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    return pd.Series({"precision": precision, "recall": recall, "f1": f1})


def calculate_overlap(row):
    intersection_size = len(row["output_set"] & row["test_set"])
    n = max(len(row["output_set"]), len(row["test_set"]))
    if n == 0: return 0
    return intersection_size / n


def calculate_jaccard(row):
    intersection_size = row['intersection_size']
    union_size = row['union_size']
    if union_size == 0: return 0
    return intersection_size / union_size

In [6]:
# Elements that differ
new_df["pred_only"] = new_df["output_set"] - new_df["test_set"]
new_df["true_only"] = new_df["test_set"] - new_df["output_set"]
# intersection size
new_df["intersection_size"] = new_df.apply(
    lambda row: len(row["output_set"] & row["test_set"]), axis=1
)
# union size
new_df["union_size"] = new_df.apply(
    lambda row: len(row["output_set"] | row["test_set"]), axis=1
)
# overlap = intersection / larger_set
new_df["overlap"] = new_df.apply(calculate_overlap, axis=1)
# Jaccard similarity (intersection / union)
new_df["jaccard"] = new_df.apply(calculate_jaccard, axis=1)
# Precision, recall, F1
new_df[["precision", "recall", "f1"]] = new_df.apply(precision_recall_f1_per_row, axis=1)

In [7]:
new_df.head(100)

,id,ingredients,output_set,test_set,pred_only,true_only,intersection_size,union_size,overlap,jaccard,precision,recall,f1
0,0,"['1 c. firmly packed brown sugar', '1/2 c. eva...","{brown sugar, evaporated milk, pecan, butter, ...","{brown sugar, evaporated milk, pecan, margarin...",{shredded rice biscuit},"{margarine, rice biscuit}",5,8,0.714286,0.625000,0.833333,0.714286,0.769231
1,1,"['1 small jar chipped beef, cut up', '4 boned ...","{chipped beef, cream of mushroom soup, chicken...","{chipped beef, cream of mushroom soup, chicken...",{},{},4,4,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","{cream cheese, salt, pepper, corn, butter, gar...","{cream cheese, salt, corn, butter, black peppe...",{pepper},{black pepper},5,7,0.833333,0.714286,0.833333,0.833333,0.833333
3,3,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","{mushroom soup, chicken gravy, stuffing, chick...","{cream of mushroom soup, chicken gravy, stuffi...","{mushroom soup, chicken}","{cream of mushroom soup, whole chicken}",3,7,0.600000,0.428571,0.600000,0.600000,0.600000
4,4,"['1 c. peanut butter', '3/4 c. graham cracker ...","{peanut butter, butter, powdered sugar, graham...","{graham cracker, peanut butter, chocolate chip...","{graham cracker crumbs, chocolate chips}","{graham cracker, chocolate chip}",3,7,0.600000,0.428571,0.600000,0.600000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,"['3 (1 lb.) cans pork and beans', '1/2 c. bell...","{brown sugar, salt, ground beef, pork and bean...","{brown sugar, salt, ground beef, pork and bean...",{},{},9,9,1.000000,1.000000,1.000000,1.000000,1.000000
96,96,"['2 c. diced cooked chicken', '2 Tbsp. shorten...","{brown sugar, shortening, cornstarch, green pe...","{brown sugar, shortening, cornstarch, green pe...",{},{},14,14,1.000000,1.000000,1.000000,1.000000,1.000000
97,97,"['8 to 10 juicy oranges, peeled and diced', '1...","{orange juice, sugar, orange, pecan, coconut, ...","{orange juice, sugar, orange, pecan, coconut, ...",{},{},6,6,1.000000,1.000000,1.000000,1.000000,1.000000
98,98,"['1 c. creamy peanut butter', '1 c. sugar', '1...","{egg, sugar, peanut butter}","{egg, sugar, peanut butter}",{},{},3,3,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
print(f"Overlap:  {new_df['overlap'].mean():.4f}")
print(f"Jaccard:   {new_df['jaccard'].mean():.4f}")
print(f"Precision: {new_df['precision'].mean():.4f}")
print(f"Recall:    {new_df['recall'].mean():.4f}")
print(f"F1:        {new_df['f1'].mean():.4f}")

Overlap:  0.8625
Jaccard:   0.7944
Precision: 0.8729
Recall:    0.8637
F1:        0.8679
